In [ ]:
import sys
import torch
import yaml

import pyrootutils
import gradio as gr

root = pyrootutils.setup_root(__vsc_ipynb_file__, dotenv=True, pythonpath=True)
sys.path.append('/data/deeplearning-all-in-one/src')

from src.models.dist_classify import DistClassify

In [ ]:
path = torch.load('/data/deeplearning-all-in-one/logs/train/runs/2022-08-03/18-17-12/checkpoints/model.ckpt')

In [ ]:
map_location = {'cuda:1'}

In [ ]:
model = DistClassify.load_from_checkpoint(path, hparams=yaml.safe_load(open('/data/deeplearning-all-in-one/logs/train/runs/2022-08-03/18-17-12/.hydra/config.yaml', 'r')), map_location=map_location)

In [ ]:
from transformers import BertTokenizer

In [ ]:
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'
model = model.to(device)
model.eval()
tokenizer = BertTokenizer.from_pretrained('hfl/chinese-roberta-wwm-ext', cache_dir='/data/.cache')

In [ ]:
label = open('/data/clean_raw_text/street_label_map.txt', 'r').readlines()

In [ ]:
label = [l.strip().split(' ')[0] for l in label]

In [ ]:
def fn(text: str):
    encoded = tokenizer([text], return_tensors='pt', truncation=True, max_length=200).to(device)
    with torch.no_grad():
        logit = model(encoded)[0] # (num_classes)
    prob = torch.nn.functional.softmax(logit, dim=0)
    top_5 = torch.topk(prob, 5).indices.tolist()
    top_5_prob = prob[top_5].tolist()
    label_top_5 = [label[i] for i in top_5]
    return {k: v for k, v in zip(label_top_5, top_5_prob)}

In [ ]:
fn('门禁开不了,请问维修电话,谢谢')

In [ ]:
demo = gr.Interface(fn, inputs='text', outputs='label')
demo.launch(share=True, debug=True, server_port=6006)

In [ ]:
demo.close()